In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset_completo = pd.read_csv("https://cdn.buenosaires.gob.ar/datosabiertos/datasets/salud/casos-covid-19/casos_covid19.csv")

C:\Users\44594722\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
dataset_completo_caba = dataset_completo[dataset_completo["provincia"]=="CABA"].copy()
dataset_completo_caba["mes_confirmado"] = dataset_completo_caba["fecha_clasificacion"].str[2:5]
dataset_completo_caba["dia_confirmado"] = dataset_completo_caba["fecha_clasificacion"].str[:2]
meses_replace = {"JAN":1,"FEB":2,"MAR":3,"APR":4,"MAY":5,"JUN":6,"JUL":7,"AUG":8,"SEP":9,"OCT":10,"NOV":11,"DEC":12}
dataset_completo_caba["mes_confirmado_num"] = dataset_completo_caba["mes_confirmado"].replace(meses_replace)
dataset_completo_caba = dataset_completo_caba.sort_values(["mes_confirmado_num", "dia_confirmado"])
dataset_completo_caba["mes_confirmado_num"] = dataset_completo_caba["mes_confirmado_num"].astype(str)
dataset_completo_caba["id_dia"] = dataset_completo_caba["dia_confirmado"].map(str) +"/"+dataset_completo_caba["mes_confirmado_num"].map(str)

In [12]:
dataset_completo_caba_confirmados = dataset_completo_caba[dataset_completo_caba["clasificacion"]=="confirmado"].copy()

In [13]:
dataset_completo_caba_muertes = dataset_completo_caba_confirmados.copy()[dataset_completo_caba_confirmados["fallecido"]=="si"]

In [14]:
dataset_completo_caba_muertes["mes_muerte"] = dataset_completo_caba_muertes["fecha_fallecimiento"].str[2:5]
dataset_completo_caba_muertes["dia_muerte"] = dataset_completo_caba_muertes["fecha_fallecimiento"].str[:2]
meses_replace = {"JAN":1,"FEB":2,"MAR":3,"APR":4,"MAY":5,"JUN":6,"JUL":7,"AUG":8,"SEP":9,"OCT":10,"NOV":11,"DEC":12}
dataset_completo_caba_muertes["mes_muerte_num"] = dataset_completo_caba_muertes["mes_muerte"].replace(meses_replace)
dataset_completo_caba_muertes = dataset_completo_caba_muertes.sort_values(["mes_muerte_num", "dia_muerte"])
dataset_completo_caba_muertes["mes_muerte_num"] = dataset_completo_caba_muertes["mes_muerte_num"].astype(str)
dataset_completo_caba_muertes["id_dia_muerte"] = dataset_completo_caba_muertes["dia_muerte"].map(str) +"/"+dataset_completo_caba_muertes["mes_muerte_num"].map(str)

In [15]:
def nuevos_casos_caba_dia(dia):
    cant_casos = dataset_completo_caba_confirmados[dataset_completo_caba_confirmados["id_dia"]==dia]
    return {dia: len(cant_casos)}

In [20]:
def nuevas_muertes_caba_dia(dia):
    cant_muertes = dataset_completo_caba_muertes[dataset_completo_caba_muertes["id_dia_muerte"]==dia]
    return {dia: len(cant_muertes)}

In [29]:
def nuevos_casos_caba_rango(desde="16/2",hasta = None):
    dict_cant_casos = {}
    in_range = False
    for i in dataset_completo_caba_confirmados["id_dia"]:
        if i == desde:
            in_range = True
        if i == hasta:
            in_range = False
        if in_range:
            dict_cant_casos[i]=dict_cant_casos.get(i,0)+1
    return dict_cant_casos

In [32]:
def nuevas_muertes_caba_rango(desde="07/3",hasta = None):
    dict_cant_muertes = {}
    in_range = False
    for i in dataset_completo_caba_muertes["id_dia_muerte"]:
        if i == desde:
            in_range = True
        if i == hasta:
            in_range = False
        if in_range:
            dict_cant_muertes[i]=dict_cant_muertes.get(i,0)+1
    return dict_cant_muertes

In [33]:
nuevos_casos_caba_dia("05/11")

{'05/11': 486}

In [34]:
nuevas_muertes_caba_dia("06/11")

{'06/11': 2}

In [35]:
nuevos_casos_caba_rango("01/11",None)

{'01/11': 274,
 '02/11': 416,
 '03/11': 463,
 '04/11': 360,
 '05/11': 486,
 '06/11': 122}

In [36]:
nuevas_muertes_caba_rango("01/11",None)

{'01/11': 13, '02/11': 8, '03/11': 10, '04/11': 6, '05/11': 5, '06/11': 2}

5221